In [4]:
import time
import json
import requests
from lxml import html
from urllib.parse import urljoin
from tqdm import tqdm
import re

def fetch_all_interviews(player_url, pause=1.0):
    resp = requests.get(player_url)
    resp.raise_for_status()
    tree = html.fromstring(resp.content)

    # Grab interview links
    raw_links = tree.xpath('//table[1]//tr/td[3]/b/a')[:]
    interviews = [{
        'title': a.text_content().strip(),
        'url': urljoin(player_url, a.get('href').strip())
    } for a in raw_links if a.get('href')]

    results = []

    for iv in tqdm(interviews, desc="Processing interviews"):
        try:
            r = requests.get(iv['url'])
            r.raise_for_status()
            page = html.fromstring(r.content)

            # Extract date
            date = page.xpath('string(//h2)').strip() or "Unknown Date"
            
            questions = {}
            answers = {}

            # First attempt: Normal XPath parsing
            container = page.xpath('//td[@valign="top"]')
            if container:
                container = container[0]
                question_nodes = container.xpath('.//*[contains(., "Q.") or starts-with(normalize-space(), "Q.")]')

                idx = 0
                for i, node in enumerate(question_nodes):
                    question_text = node.text_content().replace('Â', '').replace('\xa0', ' ').strip()
                    if question_text.startswith('Q. '):
                        question_text = question_text[3:].strip()

                    answer_parts = []
                    current_node = node.getnext()
                    next_q = question_nodes[i+1] if (i+1) < len(question_nodes) else None

                    while current_node is not None and current_node != next_q:
                        if isinstance(current_node, html.HtmlElement):
                            txt = current_node.text_content().replace('Â', '').strip()
                            if txt and not txt.startswith('Q.'):
                                answer_parts.append(txt)
                        current_node = current_node.getnext()

                    if not answer_parts:
                        node_text = node.text_content().replace('Â', '').split('Q.')[-1].strip()
                        if node_text:
                            answer_parts.append(node_text)

                    questions[str(idx)] = f"Q. {question_text}"
                    answers[str(idx)] = ' '.join(answer_parts).strip()
                    idx += 1

            # Fallback: Smart recovery parsing
            if not questions:
                raw_text = page.xpath('//pre//text() | //td[@valign="top"]//text()')
                lines = [line.strip() for line in raw_text if line.strip()]

                idx = 0
                current_question = None
                current_answer = []

                for line in lines:
                    if re.match(r'^Q\.', line) or re.match(r'^[A-Z\s]+:$', line):
                        if current_question is not None:
                            questions[str(idx)] = current_question
                            answers[str(idx)] = ' '.join(current_answer).strip()
                            idx += 1
                            current_answer = []

                        if line.startswith('Q.'):
                            current_question = line
                        else:
                            current_question = line
                    else:
                        if current_question is not None:
                            current_answer.append(line)

                if current_question is not None:
                    questions[str(idx)] = current_question
                    answers[str(idx)] = ' '.join(current_answer).strip()

            results.append({
                'title': iv['title'],
                'url': iv['url'],
                'date': date,
                'qa': {
                    'questions': questions,
                    'answers': answers
                }
            })
            
            # ✅ Optional: After each interview processed, print the title and date
            tqdm.write(f"✔️ Finished: {iv['title']} - {date}")

            time.sleep(pause)

        except Exception as e:
            tqdm.write(f"❌ Failed to process {iv['url']}: {e}")

    return results

if __name__ == '__main__':
    transcripts = fetch_all_interviews('http://www.asapsports.com/show_player.php?id=13888')
    with open('interviews.json', 'w', encoding='utf-8') as f:
        json.dump(transcripts, f, indent=2, ensure_ascii=False)
    print("\n✅ All transcripts saved to interviews.json")


Processing interviews:   0%|          | 0/249 [00:00<?, ?it/s]

✔️ Finished: NBA ALL-STAR GAME - February 16, 2025


Processing interviews:   0%|          | 1/249 [00:01<04:47,  1.16s/it]

✔️ Finished: ALL-STAR GAME - February 18, 2024


Processing interviews:   1%|          | 2/249 [00:02<04:59,  1.21s/it]

...

✔️ Finished: NBA ALL-STAR WEEKEND - February 20, 2005


Processing interviews: 100%|█████████▉| 248/249 [04:48<00:01,  1.15s/it]

✔️ Finished: NBA ALL-STAR WEEKEND - February 18, 2005


Processing interviews: 100%|██████████| 249/249 [04:49<00:00,  1.16s/it]


✅ All transcripts saved to interviews.json


In [8]:
import json
import re

# Load the original interviews.json
with open('interviews.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

def split_q_and_a(full_text):
    """Split a block into (question, answer) based on first ALL CAPS speaker (keep only last name)."""
    match = re.search(r'([A-Z][A-Z\s]+?):', full_text)
    if match:
        speaker_full = match.group(1)  # e.g., "LEBRON JAMES"
        split_idx = match.start()
        question = full_text[:split_idx].strip()
        answer = full_text[split_idx:].strip()

        # Extract only last word (last name)
        last_name = speaker_full.strip().split()[-1]
        answer = re.sub(re.escape(speaker_full + ':'), last_name + ':', answer, count=1)
    else:
        # fallback: no speaker found
        question = full_text.strip()
        answer = ""
    return question, answer

# Go through each interview
for interview in data:
    questions = interview['qa']['questions']
    answers = interview['qa']['answers']

    # Check if the questions are all just "Q."
    if all(q.strip() == "Q." for q in questions.values()):
        new_questions = {}
        new_answers = {}

        for idx, full_text in answers.items():
            if full_text.strip():
                question_part, answer_part = split_q_and_a(full_text)
                new_questions[idx] = f"Q. {question_part}"
                new_answers[idx] = answer_part
            else:
                new_questions[idx] = "Q."
                new_answers[idx] = ""

        # Update the interview entry
        interview['qa']['questions'] = new_questions
        interview['qa']['answers'] = new_answers

# Save back into interviews.json
with open('interviews.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print("✅ Fixed interviews saved to interviews.json (last name speaker format)")


✅ Fixed interviews saved to interviews.json (last name speaker format)
